## Ben Johann (bmj7sf@virginia.edu)
## DS 5001
## 6 May 2022

In [1]:
import pandas as pd
import numpy as np
import gzip
import json
import re
import nltk
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, TfidfTransformer

In [3]:
ngram_range = (1,4)
n_terms = 5000

In [2]:
data_home = 'data'
OHCO = ['main_genre','review_num','sent_num','token_num']

In [3]:
def parse(path):
  g = gzip.open(path, 'rb')
  for l in g:
    yield eval(l)

def getDF(path):
  i = 0
  df = {}
  for d in parse(path):
    df[i] = d
    i += 1
  return pd.DataFrame.from_dict(df, orient='index')

reviews = getDF(data_home+'/reviews_CDs_and_Vinyl_5.json.gz')

In [4]:
reviews.reset_index(inplace=True)
reviews.rename({'index':'review_num'}, axis=1, inplace=True)

In [5]:
reviews.loc[484, 'reviewText']

"I'm probably dead center in the target age range for this disc.  I was 14 when these shows aired, and it was definitely Must See TV for me then.  When I cracked this open today and popped it in the machine, the opening theme music and cheesy dance routines took me back 35 years in an instant.  I'm not sure I would recommend this to anyone who didn't live through it as I did, however.  My present S.O. is too young, by about 10 years, and has no recollection of this show, and frankly doesn't see why I got so excited about it.  It's hard to explain, I guess, why anyone would be thrilled to see grainy B&amp;W mostly lip-synched performances of extremely light-weight pop music.  Well, so be it, but thrilled I am. Some of it is _extremely_ light-weight, too:  I could do without about half the acts (Vikki Carr? Gary Lewis? Jerry Lewis?!?) but there's plenty to like, as well.My only real complaint is that the producers decided to  include entire weekly programs(6 half-hours, one hour-long) wh

In [6]:
#remove &quot; - I believe that it replace " in the json file
reviews.reviewText = reviews.reviewText.str.replace('&quot;', '')

In [7]:
CORPUS = reviews[['review_num','reviewText']]

In [8]:
LIB = getDF(data_home+'/meta_CDs_and_Vinyl.json.gz')

In [9]:
genres = ['Jazz','Rap & Hip-Hop','Classic Rock','R&B','Pop','Country','Blues', 'Classical']
mask = [(idx, genre) for idx, genre in enumerate(LIB.category.values) if any(i in genre for i in genres)]
mask_idx = [i[0] for i in mask]

In [10]:
len(LIB.iloc[mask_idx,:])

256653

In [11]:
LIB = LIB.iloc[mask_idx,:]

In [12]:
LIB['main_genre'] = [i[1] for i in mask]
LIB.main_genre = LIB.main_genre.apply((lambda row: [i for i in row if i in genres][0]))

In [14]:
#only take reviews that are bigger than 200 words
LIB = reviews.merge(LIB, on = 'asin')
LIB = LIB.loc[LIB.reviewText.str.split().str.len()>200]
LIB = LIB.drop('reviewText', axis=1).sort_values(['main_genre','unixReviewTime'])

In [16]:
#LIB.to_csv('LIB.csv')

In [18]:
LIB

,review_num,reviewerID,asin,reviewerName,helpful,overall,summary,unixReviewTime,reviewTime,category,...,rank,also_view,main_cat,similar_item,date,price,imageURL,imageURLHighRes,details,main_genre
22406,100736,A2KQO3RSLSSRHA,B0000024SD,Will Errickson,"[11, 14]",5.0,Essential and exquisite Dylan,900201600,"07 12, 1998","[CDs & Vinyl, Blues, Contemporary Blues]",...,"390,200 in CDs & Vinyl (","[B013QJ4RZK, B075865WJ9, B013QJ4I4K, B00026WU7...","<img src=""https://images-na.ssl-images-amazon....",,,$4.95,[],[],"{'Audio CD': '(October 25, 1990)', 'Number of ...",Blues
110547,373586,A21IK52XTZGKSG,B00000DGTN,Jerome Clark,"[16, 16]",5.0,the best modern American folk album of 1998,914371200,"12 23, 1998","[CDs & Vinyl, Blues, Contemporary Blues]",...,"274,524 in CDs & Vinyl (","[B00004YLHE, B01M05093H, B002H3ET96, B079B8K9P...","<img src=""https://images-na.ssl-images-amazon....",,,$9.67,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...,"{'Audio CD': '(April 1, 2008)', 'Number of Dis...",Blues
12918,74167,A3RCLDBIA8CAKR,B000001EWH,"Yiannis Psaroudis ""divastation.com""","[0, 0]",4.0,oh how the years go by.,916099200,"01 12, 1999","[CDs & Vinyl, Blues, Contemporary Blues]",...,"261,289 in CDs & Vinyl (","[B000001FWS, B000001FOV, B0000CERNY, B0006VXM7...","<img src=""https://images-na.ssl-images-amazon....",,,$6.58,[],[],"{'Audio CD': '(August 26, 1997)', 'Number of D...",Blues
103139,345984,A3RCLDBIA8CAKR,B000007SNA,"Yiannis Psaroudis ""divastation.com""","[6, 8]",5.0,true colors.,922924800,"04 1, 1999","[CDs & Vinyl, Blues, Vocal Blues]",...,"241,261 in CDs & Vinyl (","[B000BYR9ZY, B00000I9CK, B000005KQE, B0009F2C3...","<img src=""https://images-na.ssl-images-amazon....",,,$9.55,[],[],"{'Audio CD': '(June 30, 1998)', 'Original Rele...",Blues
111210,376195,A1OPGE49BOTNR2,B00000DP1V,Lauren (lauverf@aol.com),"[6, 6]",3.0,A few catchy melodies a musical does not make...,929491200,"06 16, 1999","[CDs & Vinyl, Blues, Contemporary Blues]",...,"43,044 in CDs & Vinyl (","[B00000D9TP, 1575603535, B000005J0O, B001EOQV5...","<img src=""https://images-na.ssl-images-amazon....",,,$9.06,[],[],"{'Audio CD': '(January 5, 1999)', 'Number of D...",Blues
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
393534,1091973,A21QD3LJIKOZO0,B00GSIRMGS,Hip-Hop-Fan2786,"[0, 0]",4.0,"My Own Lane, 3.5 Stars",1401926400,"06 5, 2014","[CDs & Vinyl, Rap & Hip-Hop, Pop Rap]",...,"417,684 in CDs & Vinyl (",[B00P08LJV6],"<img src=""https://images-na.ssl-images-amazon....",,,$7.98,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...,"{'Audio CD': '(January 7, 2014)', 'Original Re...",Rap & Hip-Hop
396023,1096746,AXL0JWEZYLW1V,B00JGWKTB8,Van Leibensperger,"[0, 0]",4.0,Sweet butter on stale bread.,1401926400,"06 5, 2014","[CDs & Vinyl, Rap & Hip-Hop, Pop Rap]",...,"189,990 in CDs & Vinyl (","[B008VFVBR2, B07BZ3L75M, B075RDCYNQ, B00VM5HIC0]","<img src=""https://images-na.ssl-images-amazon....",,,$16.44,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...,"{'Audio CD': '(May 27, 2014)', 'Number of Disc...",Rap & Hip-Hop
363972,1019235,A1228DUW5ECNGX,B005E8KSHS,Perraith,"[0, 0]",4.0,"Good Life, Decent CD",1403654400,"06 25, 2014","[CDs & Vinyl, Christian, Rap & Hip-Hop]",...,"115,215 in CDs & Vinyl (","[B00NFSM45S, B00171NZAI, B07544M62V, B00KYVVK4...","<img src=""https://images-na.ssl-images-amazon....",,,$3.84,[],[],"{'Audio CD': '(April 8, 2014)', 'Original Rele...",Rap & Hip-Hop
375414,1054442,A3IYXMGNHFQWLL,B0090OPBTC,Jesse Rose,"[0, 0]",5.0,Where's My Mother****ing Dominos?,1404259200,"07 2, 2014","[CDs & Vinyl, Rap & Hip-Hop, Pop Rap]",...,"520,220 in CDs & Vinyl (","[B009F1ZYO2, B012HOEOPS, B0716SPKYW, B00R5AI4A...","<img src=""https://images-na.ssl-images-amazon....",,,$6.99,[],[],"{'Audio CD': '(October 22, 2012)', 'Number of ...",Rap & Hip-Hop


In [19]:
LIB.groupby('main_genre')['review_num'].count()

main_genre
Blues             4778
Classic Rock     19952
Classical        14993
Country           8930
Jazz              7625
Pop              38200
R&B               8371
Rap & Hip-Hop     7856
Name: review_num, dtype: int64

In [20]:
CORPUS = CORPUS.merge(LIB, on='review_num')[['main_genre','review_num','reviewText']].set_index(['main_genre','review_num'])
CORPUS.index.names = OHCO[:2]

In [21]:
SENTS = CORPUS.reviewText.apply(lambda x: pd.Series(nltk.sent_tokenize(x)))\
        .stack()\
        .to_frame('sent_str')
SENTS.index.names = OHCO[:3]

In [22]:
SENTS

sent_str
main_genre   review_num sent_num                                                   
Classic Rock 483        0         I rate this and the other Hullabaloo DVD and V...
                        1                      So, we get to see mostly kinescopes.
                        2         But, if you can get past the sub-standard vide...
                        3         The reason is that if you look closely you can...
                        4                                          It's infectious.
...                                                                             ...
Pop          1097553    22        Remember, just forget that he's a creep and si...
                        23        I'm sure with time this album will be consider...
                        24        There are no "Blurred Lines" type songs on her...
                        25        I personally don't care whether Robin and his ...
                        26        I'm just a music lover, and this is decent new...

[1895877 rows x 1 columns]

In [23]:
from tqdm import tqdm
tqdm.pandas()

TOKENS = SENTS.sent_str\
        .progress_apply(lambda x: pd.Series(nltk.pos_tag(nltk.WhitespaceTokenizer().tokenize(x))))\
        .stack()\
        .to_frame('pos_tuple')

/apps/software/standard/compiler/gcc/9.2.0/jupyter_conda/2020.11-py3.8/lib/python3.8/site-packages/tqdm/std.py:697: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel
100%|██████████| 1895877/1895877 [55:30<00:00, 569.22it/s]  


In [24]:
TOKENS.index.names = OHCO
TOKENS

pos_tuple
main_genre   review_num sent_num token_num              
Classic Rock 483        0        0              (I, PRP)
                                 1            (rate, NN)
                                 2            (this, DT)
                                 3             (and, CC)
                                 4             (the, DT)
...                                                  ...
Pop          1097553    26       6            (this, DT)
                                 7             (is, VBZ)
                                 8          (decent, JJ)
                                 9             (new, JJ)
                                 10         (music., NN)

[42140085 rows x 1 columns]

In [25]:
TOKENS['pos'] = TOKENS.pos_tuple.apply(lambda x: x[1])
TOKENS['token_str'] = TOKENS.pos_tuple.apply(lambda x: x[0])
TOKENS['term_str'] = TOKENS.token_str.str.lower()

In [26]:
TOKENS.drop('pos_tuple', axis=1, inplace=True)

In [27]:
#Removes non alpha numeric
TOKENS.term_str = TOKENS.term_str.str.replace('[^a-zA-Z]', '')

In [28]:
TOKENS = TOKENS[TOKENS.term_str != '']

In [29]:
TOKENS

pos token_str term_str
main_genre   review_num sent_num token_num                        
Classic Rock 483        0        0          PRP         I        i
                                 1           NN      rate     rate
                                 2           DT      this     this
                                 3           CC       and      and
                                 4           DT       the      the
...                                         ...       ...      ...
Pop          1097553    26       6           DT      this     this
                                 7          VBZ        is       is
                                 8           JJ    decent   decent
                                 9           JJ       new      new
                                 10          NN    music.    music

[41547906 rows x 3 columns]

In [30]:
#TOKENS.to_csv('TOKENS.csv')

In [3]:
#TOKENS = pd.read_csv('data/TOKENS.csv')